In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

/Users/kamilakolpashnikova/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)

array([ 7.82226101e-02, -4.01311256e-02,  3.86136323e-02, -1.78981383e-04,
        8.92346501e-02, -5.04591055e-02, -1.05026653e-02,  3.71055864e-02,
       -4.18714024e-02,  3.48085202e-02, -1.20702349e-02, -2.36942600e-02,
        3.87900025e-02,  1.60988253e-02,  3.50747146e-02,  3.04755149e-03,
        5.79672307e-02, -4.10627723e-02, -3.41552831e-02, -2.56396439e-02,
       -3.55264246e-02,  1.42908059e-02, -1.62799768e-02,  3.21446322e-02,
       -4.66897376e-02,  7.89185911e-02,  4.90160920e-02,  1.56760961e-02,
       -1.69110205e-02,  2.26482023e-02,  5.60206175e-02, -3.98361422e-02,
        6.77409768e-02, -1.20210182e-02,  1.12625177e-03, -1.94394626e-02,
       -2.65951231e-02,  1.06177833e-02,  1.69687625e-02,  1.13487989e-02,
       -2.97063123e-02,  5.25258295e-02, -1.41452933e-02,  4.61699590e-02,
        1.17066121e-02, -2.38053128e-02, -6.32557869e-02, -1.92042030e-02,
       -7.10595446e-03,  3.24167721e-02,  2.49617770e-02, -5.27499523e-03,
        2.01149769e-02, -

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

Q2

In [5]:
embeddings = []

for doc in documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    emb = embedding_model.encode(qa_text)
    embeddings.append(emb)


In [6]:
import numpy as np

X = np.array(embeddings)

X.shape

(948, 768)

Q3

In [7]:
v = embedding_model.encode(user_question)
scores = X.dot(v)

In [8]:
max(scores)

0.65065753

Q4

In [9]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'Star the r

Q4

In [10]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [11]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [12]:
from tqdm.auto import tqdm

relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    query = embedding_model.encode(q['question'])
    results = search_engine.search(query, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)


  0%|          | 0/1830 [00:00<?, ?it/s]

In [13]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

hit_rate(relevance_total)

0.9218579234972678

Q5

In [14]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'db1f527041ed', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'oEtTYml8Rk-P8Wv1GSahcg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
len(v)

768

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "id": {"type": "keyword"},
            "text_vector":{"type":"dense_vector","dims": 768,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "homework-index"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework-index'})

In [17]:
operations = []
for i, doc in enumerate(documents):
	doc["text_vector"] = embeddings[i].tolist()
	operations.append(doc)

In [18]:
operations[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef',
 'text_vector': [0.0616845078766346,
  -0.004774284083396196,
  0.06304009258747101,
  0.022100325673818588,
  0.07377929985523224,
  -0.010466549545526505,
  -0.008228138089179993,
  0.07933960855007172,
  0.012464253231883049,
  -0.007290362846106291,
  -0.030235808342695236,
  -0.020197929814457893,
  0.035270851105451584,
  -0.053056031465530396,
  0.022931

In [19]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [20]:
query = {
    "field" : "text_vector",
    "query_vector" :  v,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=query,source=["text","section","question","course", "id"])
res["hits"]["hits"]

[{'_index': 'homework-index',
  '_id': 'hCRYspAB2gwx6Jaw448F',
  '_score': 0.8253288,
  '_source': {'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'id': 'ee58a693'}},
 {'_index': 'homework-index',
  '_id': 'xSRYspAB2gwx6Jaw242A',
  '_score': 0.7873189,
  '_source': {'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': "Yes, even if you don't register, you're still eligi

In [21]:
res["hits"]["hits"]

[{'_index': 'homework-index',
  '_id': 'hCRYspAB2gwx6Jaw448F',
  '_score': 0.8253288,
  '_source': {'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'id': 'ee58a693'}},
 {'_index': 'homework-index',
  '_id': 'xSRYspAB2gwx6Jaw242A',
  '_score': 0.7873189,
  '_source': {'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': "Yes, even if you don't register, you're still eligi

Q6

In [25]:
relevance_total_elastic = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    query = {
        "field" : "text_vector",
        "query_vector" :  embedding_model.encode(q['question']),
        "k" : 5,
        "num_candidates" : 10000, 
    }
    results = es_client.search(index=index_name, knn=query,source=["text","section","question","course", "id"])
    relevance = [d['_source']['id'] == doc_id for d in results["hits"]["hits"]]
    relevance_total_elastic.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [26]:
relevance_total_elastic

[[False, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, True, False, False, False],
 [False, False, True, False, False],
 [True, False, False, False, False],
 [False, True, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, False, True, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, False, True, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, True, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False]

In [27]:
hit_rate(relevance_total)

0.9218579234972678

In [28]:
hit_rate(relevance_total_elastic)

0.9218579234972678